In [12]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import mpl_toolkits
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib import colors as mcolors
import numpy as np
import scipy.integrate as intgr
import matplotlib.ticker as mtick

In [13]:
plt.style.use('../include/aps.mplstyle')
mpl.rcParams["figure.figsize"] = [3.4039, 2.10373]
rm = 2.9673;

In [14]:
def normalize_psi_PIMC(psi, x):
   int_psi_square = 2*np.pi*25*intgr.simpson(y = x*psi, x = x)
   print ("Norm = " + str(int_psi_square))
   return int_psi_square


def radial_ave(ψ,X,Y):

    x = X[0,:]
    y = Y[:,0]
    N = X.shape[0]
    dx = X[1,0]-X[0,0]
    dR = np.sqrt(2*dx**2)
    dϕ = 2*π/N
    r_vals = np.arange(0, R, dR)
    ϕ_vals = np.arange(0, 2*π, dϕ)

    if len(r_vals)*len(ϕ_vals) > N**2:
        print("Warning: Oversampling")

    # Initialize data on polar grid with fill values
    fill_value = -9999.0
    data_polar = fill_value*np.ones((len(r_vals), len(ϕ_vals)))

    # Define radius of influence. A nearest neighbour outside this radius will not
    # be taken into account.
    radius_of_influence = np.sqrt(0.1**2 + 0.1**2)

    # For each cell in the polar grid, find the nearest neighbour in the cartesian
    # grid. If it lies within the radius of influence, transfer the corresponding
    # data.
    for r, row_polar in zip(r_vals, range(len(r_vals))):
        for ϕ, col_polar in zip(ϕ_vals, range(len(ϕ_vals))):
            # Transform polar to cartesian
            _x = r*np.cos(ϕ)
            _y = r*np.sin(ϕ)

            # Find nearest neighbour in cartesian grid
            d = np.sqrt((_x-X)**2 + (_y-Y)**2)
            nn_row_cart, nn_col_cart = np.unravel_index(np.argmin(d), d.shape)
            dmin = d[nn_row_cart, nn_col_cart]

            # Transfer data
            if dmin <= radius_of_influence:
                data_polar[row_polar, col_polar] = ψ[nn_row_cart, nn_col_cart]

    # Mask remaining fill values
    data_polar = np.ma.masked_equal(data_polar, fill_value)

    return r_vals, np.average(data_polar,axis=1)

In [15]:
def cc(arg):
    return mcolors.to_rgba(arg,alpha=0.6)

In [16]:
#xs = np.arange(0, 10, 0.4)
verts = []
Rs = [4.0, 5.0, 6.0, 8.0, 10.0, 12.0]
fname_arr = ["../data/Relaxation/Radial-wavefunction-ArR4.npz","../data/Relaxation/Radial-wavefunction-ArR5.npz","../data/Relaxation/Radial-wavefunction-ArR6.npz","../data/Relaxation/Radial-wavefunction-ArR8.npz","../data/Relaxation/Radial-wavefunction-ArR10.npz","../data/Relaxation/Radial-wavefunction-ArR12.npz"]
for Rv in range(len(Rs)):
    fname = fname_arr[Rv]
    R = Rs[Rv]
    """
    Lx = 2*R; Ly = 2*R; Lz = 25
    Lxnd = Lx/rm;
    Lynd = Ly/rm;
    Lznd = Lz/rm;
    nx = 72
    ny = 72
    nz = 12
    dx = Lxnd/nx
    dy = Lynd/ny
    dz = Lznd/nz

    xmin = -Lxnd/2; xmax = Lxnd/2; ymin = -Lynd/2; ymax = Lynd/2; zmin = -Lznd/2; zmax = Lznd/2;

    x = dx * np.concatenate((np.arange(-nx/2,0,1),np.arange(0,nx/2,1)))
    y = dy * np.arange(-ny/2,ny/2,1)
    z = dz * np.arange(-nz/2, nz/2,1)

    x1, x2, y1, y2, zr = np.meshgrid(x,x,y,y,z, indexing='ij')
    xrad,yrad = np.meshgrid(x,y, indexing='ij')
    with open(fname, 'rb') as f:
        psif = np.load(f)/np.sqrt((rm**5))
    #psif = np.load(f)/(dx*dy*np.sqrt(dz*rm)*rm*rm)
    print(psif.shape)
    #print(np.sum(np.abs(psif)**2)*(dx*dx*dy*dy*dz)*(rm**5))
    p = np.sum(np.abs(np.multiply(np.conj(psif),psif)),axis=(1,3))*dx*dy*rm*rm
    rho_rad = []
    π = np.pi
    for iz in range(nz):
       rval,rho = radial_ave(p[:,:,iz], xrad*rm, yrad*rm)
       rho_rad.append(rho)
    rho_rad = np.asarray(rho_rad)
    radial = np.mean(rho_rad, axis = 0)
    """
    with np.load(fname) as f:
        rval = f['arr_0']
        radial = f['arr_1']
    normalize_psi_PIMC(radial,rval)
    rval = np.insert(rval, 0, -0.1) 
    radial = np.insert(radial, 0, 0)
    
    radial[-1] = 0
    
    verts.append(list(zip(rval, radial)))

Norm = 1.9943041040885479
Norm = 2.012019301825279
Norm = 2.024150715245431
Norm = 1.9800310251346966
Norm = 2.1046567393856987
Norm = 1.9415150846487308


In [17]:
poly = PolyCollection(verts, facecolors=[cc('#D7414E'), cc('#D7414E'), cc('#D7414E') ,cc('#D7414E'),cc('#D7414E'),cc('#D7414E')])

In [18]:
#xs = np.arange(0, 10, 0.4)
verts1 = []
Rs = [4.0, 5.0, 6.0, 8.0, 10.0, 12.0]
fname_arr = ["../data/Relaxation/Radial-wavefunction-MgR4.npz","../data/Relaxation/Radial-wavefunction-MgR5.npz","../data/Relaxation/Radial-wavefunction-MgR6.npz","../data/Relaxation/Radial-wavefunction-MgR8.npz","../data/Relaxation/Radial-wavefunction-MgR10.npz","../data/Relaxation/Radial-wavefunction-MgR12.npz"]
for Rv in range(len(Rs)):
    fname = fname_arr[Rv]
    R = Rs[Rv]
    """
    Lx = 2*R; Ly = 2*R; Lz = 25
    Lxnd = Lx/rm;
    Lynd = Ly/rm;
    Lznd = Lz/rm;
    nx = 72
    ny = 72
    nz = 12
    dx = Lxnd/nx
    dy = Lynd/ny
    dz = Lznd/nz

    xmin = -Lxnd/2; xmax = Lxnd/2; ymin = -Lynd/2; ymax = Lynd/2; zmin = -Lznd/2; zmax = Lznd/2;

    x = dx * np.concatenate((np.arange(-nx/2,0,1),np.arange(0,nx/2,1)))
    y = dy * np.arange(-ny/2,ny/2,1)
    z = dz * np.arange(-nz/2, nz/2,1)

    x1, x2, y1, y2, zr = np.meshgrid(x,x,y,y,z, indexing='ij')
    xrad,yrad = np.meshgrid(x,y, indexing='ij')
    with open(fname, 'rb') as f:
        psif = np.load(f)/np.sqrt((rm**5))
    #psif = np.load(f)/(dx*dy*np.sqrt(dz*rm)*rm*rm)
    print(psif.shape)
    #print(np.sum(np.abs(psif)**2)*(dx*dx*dy*dy*dz)*(rm**5))
    p = np.sum(np.abs(np.multiply(np.conj(psif),psif)),axis=(1,3))*dx*dy*rm*rm
    rho_rad = []
    π = np.pi
    for iz in range(nz):
       rval,rho = radial_ave(p[:,:,iz], xrad*rm, yrad*rm)
       rho_rad.append(rho)
    rho_rad = np.asarray(rho_rad)
    radial = np.mean(rho_rad, axis = 0)
    """
    with np.load(fname) as f:
        rval = f['arr_0']
        radial = f['arr_1']
    normalize_psi_PIMC(radial,rval)
    rval = np.insert(rval, 0, -0.1) 
    radial = np.insert(radial, 0, 0)
    radial[-1] = 0
    verts1.append(list(zip(rval, radial)))

Norm = 2.010337365927424
Norm = 1.993634737869944
Norm = 2.0064659361227046
Norm = 2.0067059874964754
Norm = 1.958938157067764
Norm = 2.016635750803403


In [19]:
poly2 = PolyCollection(verts1, facecolors=[cc('#79C9A4'), cc('#79C9A4'), cc('#79C9A4'),cc('#79C9A4'),cc('#79C9A4'),cc('#79C9A4')])

In [20]:
#xs = np.arange(0, 10, 0.4)
verts2 = []
Rs = [4.0, 5.0, 6.0, 8.0, 10.0, 12.0]
fname_arr = ["../data/Relaxation/Radial-wavefunction-CsR4.npz","../data/Relaxation/Radial-wavefunction-CsR5.npz","../data/Relaxation/Radial-wavefunction-CsR6.npz","../data/Relaxation/Radial-wavefunction-CsR8.npz","../data/Relaxation/Radial-wavefunction-CsR10.npz","../data/Relaxation/Radial-wavefunction-CsR12.npz"]
for Rv in range(len(Rs)):
    fname = fname_arr[Rv]
    R = Rs[Rv]
    """
    Lx = 2*R; Ly = 2*R; Lz = 25
    Lxnd = Lx/rm;
    Lynd = Ly/rm;
    Lznd = Lz/rm;
    nx = 72
    ny = 72
    nz = 12
    dx = Lxnd/nx
    dy = Lynd/ny
    dz = Lznd/nz

    xmin = -Lxnd/2; xmax = Lxnd/2; ymin = -Lynd/2; ymax = Lynd/2; zmin = -Lznd/2; zmax = Lznd/2;

    x = dx * np.concatenate((np.arange(-nx/2,0,1),np.arange(0,nx/2,1)))
    y = dy * np.arange(-ny/2,ny/2,1)
    z = dz * np.arange(-nz/2, nz/2,1)

    x1, x2, y1, y2, zr = np.meshgrid(x,x,y,y,z, indexing='ij')
    xrad,yrad = np.meshgrid(x,y, indexing='ij')
    with open(fname, 'rb') as f:
        psif = np.load(f)/np.sqrt((rm**5))
    #psif = np.load(f)/(dx*dy*np.sqrt(dz*rm)*rm*rm)
    print(psif.shape)
    #print(np.sum(np.abs(psif)**2)*(dx*dx*dy*dy*dz)*(rm**5))
    p = np.sum(np.abs(np.multiply(np.conj(psif),psif)),axis=(1,3))*dx*dy*rm*rm
    rho_rad = []
    π = np.pi
    for iz in range(nz):
       rval,rho = radial_ave(p[:,:,iz], xrad*rm, yrad*rm)
       rho_rad.append(rho)
    rho_rad = np.asarray(rho_rad)
    radial = np.mean(rho_rad, axis = 0)
    """
    with np.load(fname) as f:
        rval = f['arr_0']
        radial = f['arr_1']
    normalize_psi_PIMC(radial,rval)
    rval = np.insert(rval, 0, -0.1) 
    radial = np.insert(radial, 0, 0)
    radial[-1] = 0
    verts2.append(list(zip(rval, radial)))

Norm = 2.0424958295357984
Norm = 2.0093979071313233
Norm = 1.997469735916623
Norm = 1.9989870399327152
Norm = 1.9990406105038216
Norm = 2.0072822408958575


In [21]:
poly3 = PolyCollection(verts2, facecolors=[cc('#5E4FA2'), cc('#5E4FA2'), cc('#5E4FA2'),cc('#5E4FA2'),cc('#5E4FA2'),cc('#5E4FA2')])

In [22]:
# Define the triangle vertices in 3D (z=0 means it's on the "bottom" plane)
triangle_vertices = np.array([
    [4.0, 4.0, 0.0],
    [11.0, 4.0, 0.0],
    [11.0, 12.0, 0.0]
])

# Prepare the triangle as a Poly3DCollection
vertstr = [list(triangle_vertices)]
#triangle = Poly3DCollection(vertstr, facecolor='k', edgecolor='black')

In [23]:
with plt.style.context('aps'):
    figsize = plt.rcParams['figure.figsize']
    fig= plt.figure(figsize=(2*figsize[0],figsize[1]),constrained_layout=True)
    #fig,ax = plt.subplots(figsize=(figsize[0],figsize[1]), projection='3d',constrained_layout=True)
    ax1 = fig.add_subplot(131,projection='3d')
    poly.set_alpha(0.7)
    ax1.add_collection3d(poly, zs=Rs, zdir='y')
    triangle1 = Poly3DCollection(vertstr, facecolor='#C0C0C0', edgecolor='#C0C0C0')
    ax1.add_collection3d(triangle1)
    ax1.set_xlabel(r'$r$ [Å]',labelpad=-10,size='x-small')
    ax1.set_xlim3d(-0.1, 11)
    ax1.set_ylabel(r'$R$ [Å]',labelpad=-6,size='x-small')
    ax1.set_ylim3d(4.0, 12.0)
    #ax1.set_zlabel(r'$\rho(r)$',labelpad=0)
    ax1.set_zlim3d(0, 0.0125)
    ax1.tick_params(labelsize=4,pad=-5)
    ax1.tick_params(axis='z',labelsize=4,pad=-2)
    ax1.text(3,12.0,0.007,'Argon','x',color='k')
    #ax1.zaxis.set_major_formatter(mtick.FormatStrFormatter('%.1e'))
    #ax1.set_title(r'$^4$\rm He/Argon')
    ax2 = fig.add_subplot(132,projection='3d')
    poly.set_alpha(0.7)
    triangle2 = Poly3DCollection(vertstr, facecolor='#C0C0C0', edgecolor='#C0C0C0')
    ax2.add_collection3d(poly2, zs=Rs, zdir='y')
    ax2.add_collection3d(triangle2)
    ax2.set_xlabel(r'$r$ [Å]',labelpad=-10,size='x-small')
    ax2.set_xlim3d(-0.1, 11)
    ax2.set_ylabel(r'$R$ [Å]',labelpad=-6,size='x-small')
    ax2.set_ylim3d(4.0, 12.0)
    #ax2.set_zlabel(r'$\rho(r)$',labelpad=0)
    ax2.set_zlim3d(0, 0.02)
    ax2.text(3,12.0,0.009,'Magnesium','x',color='k')
    #ax2.set_title(r'$^4$\rm He/Magnesium')
    ax2.tick_params(labelsize=4,pad=-5)
    ax2.tick_params(axis='z',labelsize=4,pad=-2)
    ax3 = fig.add_subplot(133,projection='3d')
    poly.set_alpha(0.7)
    triangle3 = Poly3DCollection(vertstr, facecolor='#C0C0C0', edgecolor='#C0C0C0')
    ax3.add_collection3d(poly3, zs=Rs, zdir='y')
    ax3.add_collection3d(triangle3)
    ax3.set_xlabel(r'$r$ [Å]',labelpad=-10,size='x-small')
    ax3.set_xlim3d(-0.1, 11)
    ax3.set_ylabel(r'$R$ [Å]',labelpad=-6,size='x-small')
    ax3.set_ylim3d(4.0, 12.0)
    #ax3.set_zlabel(r'$\rho(r)$',labelpad=0)
    ax3.set_zlim3d(0, 0.03)
    ax3.text(3,12.0,0.012,'Cesium','x',color='k')
    #ax3.set_title(r'$^4$\rm He/Cesium')
    ax3.tick_params(labelsize=4,pad=-5)
    ax3.tick_params(axis='z',labelsize=4,pad=-3)
    plt.savefig('../figures/Waterfalls.pdf')
#plt.savefig('WaterfallAr.pdf')
#plt.show()